**Чат-бот**

Чат-бот будет поддерживать диалог на следующие темы:
- диалог ("болталка")
- предлагать рецепты
- предлагать похожие отзывы на лекарства


Для того, чтобы чат-бот определял тему вопроса (интент), используем предобученную модель Bert на следующих датасетах: 
- "ответы"
- рецепты
- отзывы на лекарства 

**Загружаем библиотеки и датасеты**

In [1]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=2dc1dd5a7502be6c6e17e2253758b578bacb01b7d765ae714dbc783eead19af0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32897 sha256=afc8a18d25dcb4e093447dad82ca74c10b4dc23a8bc204e66756c2a52d61f838
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [3]:
pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582712 sha256=95f7ed7c1ecc488689efc9ec31209cbae5bb2f5435a81254247b67e756185d2f
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297
Successfully built annoy


In [4]:
import os
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [5]:
pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 3.5 MB/s eta 0:00:00


In [6]:
from corus import load_rudrec

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [8]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00


In [9]:
from transformers import BertTokenizer

In [10]:
from transformers import AutoTokenizer

In [11]:
from transformers import TFAutoModelForSequenceClassification

In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report

In [13]:
import re

In [14]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [28]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [30]:
from transformers import TFAutoModel

Подключаем гугл-диск

In [16]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [17]:
cd "/content/drive/My Drive/Geekbrains_NLP_Project/"

/content/drive/My Drive/Geekbrains_NLP_Project


In [ ]:
!ls "/content/drive/My Drive/Geekbrains_NLP_Project/"

all_recepies_inter.csv	data_new.csv		 Questions.txt
data.csv		data_rec.csv		 rudrec_annotated.json
data_dialog.csv		model_intent_classifier  rudrec_annotated.json.1
data_lec.csv		Otvety_1.txt		 rudrec_annotated.json.2


In [18]:
path = "/content/drive/My Drive/Geekbrains_NLP_Project/"

Загружаем датасет с вопросами-ответами. Оставляем вопросы.

In [ ]:
assert True

#Small preprocess of the answers

written = False

with open("Questions.txt", "w") as fout:
    with open("Otvety_1.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written:
                fout.write(line.replace("\t", " ").strip() + "\n")
                written = True
                continue

<ipython-input-256-a55f07741842>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

Сделаем из этого файла датасет data_dialog с меткой класса 0

In [ ]:
data_dialog = {'id': [], 'text': [], 'class': []}
with open(path + "Questions.txt", "r") as f:
    for idx, line in enumerate(f):
      data_dialog['id'].append(idx)
      data_dialog['text'].append(line.strip())
      data_dialog['class'].append(0)
data_dialog = pd.DataFrame(data_dialog)

In [ ]:
data_dialog.shape

(1163422, 3)

In [ ]:
data_dialog.head(5)

,id,text,class
0,0,,0
1,1,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,0
2,2,Как парни относятся к цветным линзам? Если у д...,0
3,3,"Что делать, сегодня нашёл 2 миллиона рублей? .",0
4,4,Эбу в двенашке называется Итэлма что за эбу? .,0


Загрузим датасет с рецептами, присвоим ему метку класса 1 
(https://www.kaggle.com/coolonce/recipes-and-interpretation-dim)

In [ ]:
!ls

all_recepies_inter.csv	Questions.txt	       rudrec_annotated.json.1
Otvety_1.txt		rudrec_annotated.json


In [ ]:
data_rec = pd.read_csv('all_recepies_inter.csv', sep='\t')

In [ ]:
data_rec.shape

(27884, 10)

In [ ]:
data_rec.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [ ]:
data_rec['class'] = 1

In [ ]:
data_rec.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter,class
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая...",1
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ...",1
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква...",1
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni...",1
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u...",1


Далее используем датасет с отзывами на лекарства, присвоим ему метку класса 2

In [ ]:
! wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2023-06-08 16:06:37--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-06-08 16:06:37--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json.5’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.04s   

2023-06-08 16:06:37 (47.2 MB/s) - ‘rudrec_annotated.json.5’ saved [1773014/1773014]



In [ ]:
path_2 = 'rudrec_annotated.json'
records = load_rudrec(path_2)

In [ ]:
data_lec = {'id': [], 'text': [], 'class': []}
for idx, rec in enumerate(records):

    data_lec['id'].append(idx)
    data_lec['text'].append(rec.text.strip())
    data_lec['class'].append(2)

data_lec = pd.DataFrame(data_lec)

In [ ]:
data_lec.shape

(4809, 3)

In [ ]:
data_lec.head(5)

,id,text,class
0,0,"нам прописали, так мой ребенок сыпью покрылся,...",2
1,1,Общее впечатление : не подошел,2
2,2,Пила этот препарат для повышения иммунитета 5 ...,2
3,3,"Так как начала работать в аптеке, начала часто...",2
4,4,В месяц по нескольку раз причем со всеми вытек...,2


Теперь отбросим лишние колонки в датасетах: оставим по 1 самой значимой 
по содержанию (инструкция в рецептах, в других - текст) и колонки с меткой класса. Оставим в каждом датасете по 4500 записей и сконкатенируем датасеты.

In [ ]:
data_dialog = data_dialog[['text', 'class']]

In [ ]:
data_dialog = data_dialog.sample(4500)

In [ ]:
data_dialog.shape

(4500, 2)

In [ ]:
data_rec = data_rec[['Инструкции', 'class']]

In [ ]:
data_rec = data_rec.sample(4500)

In [ ]:
data_rec.rename(columns={'Инструкции': 'text'}, inplace=True)

In [ ]:
data_rec.head(3)

,text,class
21344,1. Половину чеснока растереть с кедровыми орех...,1
23564,1. Для соуса смолоть в блендере или в ступе ке...,1
6668,1. Разогреть духовку до 200 градусов. Положить...,1


In [ ]:
data_lec = data_lec[['text', 'class']]

In [ ]:
data_lec = data_lec.sample(4500)

Объединю данные в один датафрейм с тремя классами

In [ ]:
data = pd.concat([data_dialog, data_rec, data_lec], ignore_index=True)

In [ ]:
data.head(2)

,text,class
0,ЗАДАЧИ ПО ХИМИИ!! 1. Вычислите концентрацию ио...,0
1,Синтаксический разбор предложения дождевые кап...,0


In [ ]:
data = data.sample(frac = 1)

Сохраняем датасет

In [ ]:
data.to_csv(path + 'data.csv', index=False)

In [ ]:
data_dialog.to_csv(path + 'data_dialog.csv', index=False)
data_rec.to_csv(path + 'data_rec.csv', index = False)
data_lec.to_csv(path + 'data_lec.csv', index = False)

**Используем предобученную модель BERT для классификации темы (интента)**


In [ ]:
text_train, text_test, y_train, y_test = train_test_split(data['text'], 
                                                            data['class'], 
                                                            test_size=0.3, 
                                                            shuffle=True,
                                                            stratify=data['class'],
                                                            random_state = 1)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
tokenizer_classifier = AutoTokenizer.from_pretrained("bert-base-multilingual-cased") #для перезапуска кода, когда уже есть обученная модель

In [ ]:
tf_train = tokenizer(text_train.tolist(), padding="max_length", truncation=True, return_tensors='tf')
tf_valid = tokenizer(text_test.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [ ]:
num_classes = 3
tf_y_train = tf.keras.utils.to_categorical(y_train.tolist(), num_classes)
tf_y_valid = tf.keras.utils.to_categorical(y_test.tolist(), num_classes)

In [ ]:
train_features = {x: tf_train[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_y_train))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train)).batch(4)

valid_features = {x: tf_valid[x] for x in tokenizer.model_input_names}
valid_tf_dataset = tf.data.Dataset.from_tensor_slices((valid_features, tf_y_valid))
valid_tf_dataset = valid_tf_dataset.batch(4)

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 177,855,747
Trainable params: 177,855,747
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[0].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.CategoricalAccuracy('accuracy')],
)

Обучаем модель (верхний слой)

In [ ]:
model.layers[0].trainable = False
model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=1)

2363/2363 [==============================] - 651s 275ms/step - loss: 0.2629 - accuracy: 0.9168 - val_loss: 0.1593 - val_accuracy: 0.9563


In [ ]:
model.layers[0].trainable = False
model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=4)

Epoch 1/4
2363/2363 [==============================] - 651s 275ms/step - loss: 0.2241 - accuracy: 0.9238 - val_loss: 0.1351 - val_accuracy: 0.9580
Epoch 2/4
2363/2363 [==============================] - 651s 275ms/step - loss: 0.2118 - accuracy: 0.9249 - val_loss: 0.1279 - val_accuracy: 0.9600
Epoch 3/4
2363/2363 [==============================] - 648s 274ms/step - loss: 0.2003 - accuracy: 0.9295 - val_loss: 0.1157 - val_accuracy: 0.9630
Epoch 4/4
2363/2363 [==============================] - 652s 276ms/step - loss: 0.1880 - accuracy: 0.9307 - val_loss: 0.1236 - val_accuracy: 0.9588


Сохраняем модель

In [ ]:
model.save_pretrained(path + 'model_intent_classifier/')

Тестируем модель


In [20]:
model_test = TFAutoModelForSequenceClassification.from_pretrained(path + 'model_intent_classifier/')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/My Drive/Geekbrains_NLP_Project/model_intent_classifier/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
val_predict = model_test.predict(valid_tf_dataset).logits.argmax(axis=1)

1013/1013 [==============================] - 187s 175ms/step


In [ ]:
print(classification_report(y_test, val_predict))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1350
           1       0.99      0.99      0.99      1350
           2       0.96      0.91      0.93      1350

    accuracy                           0.96      4050
   macro avg       0.96      0.96      0.96      4050
weighted avg       0.96      0.96      0.96      4050



In [ ]:
def intent_classifier(sentence):
    sent_token = tokenizer_classifier(sentence, padding="max_length", truncation=True, return_tensors='tf')
    predict = model_test(sent_token).logits.numpy()
    return predict.argmax()

In [ ]:
intent_classifier('Добрый день!') 

2

In [ ]:
intent_classifier('Добрый день!Что можно сделать из муки и яиц?')

0

In [ ]:
intent_classifier('привет')

2

In [ ]:
intent_classifier('привет! Как дела? Что делаешь?')

0

In [ ]:
intent_classifier('Помидоры, сыр, хлеб и огурцы, понадобится духовка')

2

In [ ]:
intent_classifier('Помидоры, сыр, хлеб и огурцы, смешать, поджарить')

1

Вывод: плохо определяет кулинарную категорию, реагирует на глаголы. Видимо, надо изменить кулинарный датасет, добавив туда не только инструкции, и доубучить модель (при наличии времени и ресурсов)

**Создание моделей для интентов**

**Болталка**

Используем предобученную модель Сбербанка

In [21]:
tokenizer_talk = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
model_talk = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

**Поиск похожих отзывов на лекарства**

При помощи предобученной модели Bert и библиотеки Annoy будем находить максимально похожие высказывания из датасета отзывов на лекарства

In [25]:
pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.1 MB/s eta 0:00:00


In [26]:
import dill

In [31]:
from tqdm import tqdm_notebook
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
data_lec.head()

,id,text,class
0,0,"нам прописали, так мой ребенок сыпью покрылся,...",2
1,1,Общее впечатление : не подошел,2
2,2,Пила этот препарат для повышения иммунитета 5 ...,2
3,3,"Так как начала работать в аптеке, начала часто...",2
4,4,В месяц по нескольку раз причем со всеми вытек...,2


In [ ]:
data_lec.shape

(4809, 3)

In [ ]:
data_lec_part = data_lec.sample(2000)

Пришлось сократить список, так как модель постоянно ошибалась

In [ ]:
data_lec_part.to_csv(path + 'data_lec_part.csv', index=False) #фиксируем 

In [42]:
data_lec_part = pd.read_csv('data_lec_part.csv')

In [ ]:
data_lec_part.text.apply(lambda x: len(str(x))).sort_values(ascending=False)[:10]

1511    592
1376    531
1003    523
1433    500
1048    479
368     430
1337    394
1710    366
735     355
1898    313
Name: text, dtype: int64

In [44]:
data_lec_part_tl = data_lec_part.text.tolist()

In [32]:
bert_lec = TFAutoModel.from_pretrained("setu4993/LaBSE")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [33]:
tokenizer_lec = AutoTokenizer.from_pretrained("setu4993/LaBSE")

In [ ]:
len_data_lec_part = len(data_lec_part)

In [ ]:
encodings_lec_1 = tokenizer_lec(data_lec_part[:len_data_lec_part//2], 
                      max_length=512, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')
encodings_lec_2 = tokenizer_lec(data_lec_part[len_data_lec_part//2:], 
                      max_length=512, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')

In [ ]:
dataset_lec_1 = tf.data.Dataset.from_tensor_slices(dict(encodings_lec_1)).batch(4)
dataset_lec_2 = tf.data.Dataset.from_tensor_slices(dict(encodings_lec_2)).batch(4)

In [ ]:
out_lec_1 = bert_lec.predict(dataset_lec_1, batch_size=4).pooler_output

250/250 [==============================] - 47s 173ms/step


In [ ]:
out_lec_2 = bert_lec.predict(dataset_lec_2, batch_size=4).pooler_output

250/250 [==============================] - 43s 172ms/step


In [ ]:
out_lec = np.concatenate([out_lec_1, out_lec_2])

In [ ]:
with open (path + 'out_lec_part.dill', 'wb') as file:
    dill.dump(out_lec, file)

In [34]:
with open(path + 'out_lec_part.dill', 'rb') as f:
    vectors = dill.load(f)

In [35]:
def get_annoy_index(embeddings):
    index = annoy.AnnoyIndex(768 ,'angular')
    
    counter = 0

    for vector in tqdm_notebook(embeddings):
        index.add_item(counter, vector)
        counter += 1
        
    index.build(15, n_jobs=-1)

    return index

In [37]:
index = get_annoy_index(vectors)

<ipython-input-35-42d20ee12c26>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for vector in tqdm_notebook(embeddings):


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
index.save(path + 'index_lec_part.ann')

True

Проверим работу поиска

In [45]:
tokens = tokenizer_lec('Укрепить иммунитет', 
                      max_length=512, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')

In [46]:
vector_b = bert_lec(**tokens).pooler_output[0]

In [47]:
answers = index.get_nns_by_vector(vector_b, 3)
answers

[192, 1707, 1215]

In [91]:
answer = index.get_nns_by_vector(vector_b, 1)
answer

[192]

In [48]:
data_lec_part_tl[192]

'К тому же еще и иммунитет повышают.'

In [49]:
data_lec_part_tl[1215]

'У ребенка мне кажется уже нет своего иммунитета, она у них болеет очень часто.'

**Рецепты**

Вызывается википедия

In [105]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=548ec8ec0014d370a8251edc58c485081310d7586603516140bb92eee836eed2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [106]:
import wikipedia
import re
wikipedia.set_lang("ru")

In [107]:
# Чистим текст статьи из Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:2000]
        # Разбиваем, считая точку разделителем
        wikimas=wikitext.split('.')
        # Отбрасываем всё после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
                    # Если в строке осталось больше трех символов, добавляем ее
                    # к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'В энциклопедии нет информации об этом'

In [ ]:
print(getwiki('фаршированные томаты'))

Фаршированные помидоры — традиционная закуска и гарнир из помидоров в различных странах Европы , Ближнего Востока и Южной Америки.
При фаршировании помидоров внутренняя часть удаляется и плод начиняется фаршем из других продуктов. Для начинки используется сыр, мясо, рис, морепродукты, грибы и овощи. Фаршированные помидоры можно доготавливать, запекая их в духовке, получая таким образом, горячую закуску; или оставить помидоры сырыми с уже готовой к употреблению начинкой, получая холодную закуску. Во втором случает помидоры часто наполняют салатами.


In [ ]:
print(getwiki('рассольник с перловкой'))

Рассо́льник  — один из основных видов горячих заправочных супов в русской кухне, следующий по значимости за борщом и щами. Русское национальное блюдо с острым кисловатым вкусом и пряно-овощным ароматом, отличающееся многообразием ингредиентов. Ароматный рассольник представляет собой овоще-крупяной суп густой консистенции на основе перловой крупы или риса, обязательными ингредиентами которого, помимо субпродуктов , являются солёные огурцы, огуречный рассол и коренья. В русской традиции к рассольнику подают расстегаи с рыбой, ватрушки с творогом и пресные слоёные пирожки с ливером.
Суп рассольник не упоминается в русских кулинарных книгах до середины XVIII века, но его предшественником считается суп калья из курятины, рыбы и паюсной икры не только с солёными огурцами, но и с солёными лимонами и на лимонном растворе. В старинной русской кухне широко использовались разнообразные рассолы. Помимо кальи и солянки с огуречным рассолом, на Руси готовили также кислые и острые супы-похмелки с соо

**Собираем бота**

Классификация входящих сообщений:

In [50]:
def intent_classifier(sentence):  #эта функция уже была использована в начале блокнота для тестирования берта-классификатора
    sent_token = tokenizer_classifier(sentence, padding="max_length", truncation=True, return_tensors='tf')
    predict = model_test(sent_token).logits.numpy()
    return predict.argmax()

"Болталка":

In [143]:
def get_answer(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer_talk(prefix, return_tensors='pt')
    tokens = {k: v for k, v in tokens.items()}
    end_token_id = tokenizer_talk.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model_talk.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer_talk.eos_token_id
    )
    decoded = tokenizer_talk.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [144]:
def talk_generator(sentence):
    history = [sentence]
    answer = get_answer(history[-10:])
    return answer

Подсказка похожих отзывов на лекарства:

In [102]:
def lec_recom(sentence):
    tokens = tokenizer_lec(sentence,
                           max_length=512,
                           truncation=True,
                           padding='max_length',
                           return_token_type_ids=False,
                           return_tensors='tf')
    vector = bert_lec(**tokens).pooler_output[0]
    index_val = index.get_nns_by_vector(vector, 1)
    res = data_lec_part['text'].iloc[index_val].tolist()
    return res

In [103]:
lec_recom('Укрепить иммунитет')

['К тому же еще и иммунитет повышают.']

Вызов информации о продукте/рецепте на кулинарную тему смотреть выше.

In [148]:
def get_response(sentence):
      predict_intent = intent_classifier(sentence)
      if predict_intent == 0:
          res = talk_generator(sentence)
      elif predict_intent == 1:
          res = lec_recom(sentence)
      elif predict_intent == 2:
          res = getwiki(sentence)
      return res

In [149]:
get_response('укрепить иммунитет')

'Иммуномодуляторы — разнородные биологически активные вещества, влияющие на иммунную систему либо участвующие в её функционировании. Не оказывают прямого воздействия на патогены, но могут изменять иммунный ответ клеток. Иммуномодулирующие препараты обычно используются для регулирования или нормализации работы иммунной системы, в частности, при лечении рака, инфекций, наследственного ангионевротического отёка  и других заболеваний, а также в качестве поддерживающей терапии при астме. По характеру влияния иммуномодулирующие препараты подразделяют на иммуностимулирующие и иммуносупрессивные.\nВ России и в западных странах под иммуномодуляторами могут понимать различные группы препаратов. На российском рынке представлено около сотни иммуномодуляторов, разработанных внутри страны и практически не использующихся за пределами СНГ. В России эта группа веществ имеет значительные объёмы продаж , уступающие лишь антибиотикам и онкопрепаратам, при этом ряд препаратов, позиционируемых производителя

In [145]:
get_response('Как жить без борща?')

'В чем смысл жизни?'

In [150]:
get_response('суп-пюре')

'паштет'

In [151]:
get_response('суп-пюре нарезать смешать подогреть')

['Нужно использовать комплексное лечение, прогревание и принимать лекарства от кашля и насморка отдельно.']